# Compare diffraction data across different scans

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

Change to HDF5 server to load all modules
DanMAX.py Version 3.0.0


In [7]:
import lib.mapping as mapping

ImportError: cannot import name 'colorCylinderAnnotation' from 'texture' (unknown location)

### Import data

In [6]:
# define scans to include in the comparison
scans = [400]

x,I,t,T,I0,E = {},{},{},{},{},{}
for scan in scans:
    # find scan path
    fname = DM.findScan(scan)
    # get the azimuthally integrated filename from master file name
    aname = DM.getAzintFname(fname)
    if scan == 601:
        aname = aname2
    # read the integrated data
    data = DM.getAzintData(aname)
    if type(data['q']) != type(None):
        x[scan] = data['q']
        Q = True
    else:
        x[scan] = data['tth']
        Q = False
    
    # read common meta data from the master file
    meta = DM.getMetaData(fname)
    print(meta['I0'].shape)
    # reduce time-resolution to speed up initial analysis
    rf = 1
    start = None 
    end =  None
    data = DM.reduceDic(data,reduction_factor=rf,start=start,end=end)
    meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)
    
    I[scan] = data['I']
    #meta_dic = DM.getMetaDic(fname) # extended meta data dictionary
    t[scan] = meta['time'] # relative time stamp in seconds
    T[scan] = meta['temp'] # temperature in Kelvin (if available, otherwise None)
    I0[scan]= meta['I0']  # relative incident beam intensity "I zero"
    E[scan] = meta['energy'] # X-ray energy in keV

    print(f'Effective time-resolution: {np.mean(np.diff(t[scan])):.2f} s')

    # normalize the integrated intensity to I0
    I[scan] = (I[scan].T/I0[scan]).T

No lakeshore temperature available
No cryo stream temperature available
(91,)
Effective time-resolution: 1.99 s


### Plot average diffraction patterns

In [ ]:
# initialize figure
plt.figure()
plt.title('Pattern comparison')
if Q:
    plt.xlabel(r'Q [$\AA^{-1}$]')
else:
    plt.xlabel(r'2$\theta$ [$\deg$]')
plt.ylabel('I [a.u.]')
y_avg = {}
for scan in scans:
    y_avg[scan] = np.mean(I[scan],axis=0)
    y_avg[scan] /= np.max(y_avg[scan])
    plt.plot(x[scan],y_avg[scan],'.-',label=f'scan-{scan:04d}')
plt.legend()